This is a tutorial to use the flip package: https://github.com/corentinravoux/flip \
It is self-contained and can be used in google collab or on your environement \
All the data used are subsampled version of a simulation. \
The data size is small for the tutorial, do not use it for science case. \

In [ ]:
%%capture
!pip install git+https://github.com/corentinravoux/flip

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from flip import utils, fitter, plot_utils, data_vector, __flip_dir_path__
from flip.covariance import covariance, contraction
import flip
flip_base = Path(__flip_dir_path__)
data_path = flip_base / "data"
plt.style.use(data_path / "style.mplstyle")

[ 000000.00 ]: 11-24 15:33  root            WARNING  Install pytorch to use the nnmatrix emulator
[ 000000.00 ]: 11-24 15:33  root            INFO     No pypower module detected, gridding with this method is unavailable
[ 000000.00 ]: 11-24 15:33  root            WARNING  Install CLASS https://github.com/lesgourg/class_public to use class_engine.py module
[ 000000.00 ]: 11-24 15:33  root            WARNING  Install CCL https://github.com/LSSTDESC/CCL to use pyccl_engine.py module
[ 000000.00 ]: 11-24 15:33  root            INFO     Jax is not available, loading numpy and scipy instead


In [2]:
flip.covariance.emulators.gpmatrix.gpy_installed

True

Loading the data, located in the package itself



In [3]:
data_velocity = pd.read_parquet(data_path / "velocity_data.parquet")


data_velocity_true = {**data_velocity.rename(columns={'vpec': 'velocity'}).to_dict(orient='list'), 
                      "velocity_error": np.zeros(len(data_velocity["vpec"].values))}


# Create the data Vector
DataTrueVel = data_vector.DirectVel(data_velocity_true)

ktt, ptt = np.loadtxt(data_path / "power_spectrum_tt.txt")
kmt, pmt = np.loadtxt(data_path / "power_spectrum_mt.txt")
kmm, pmm = np.loadtxt(data_path / "power_spectrum_mm.txt")

sigmau_fiducial = 15

power_spectrum_dict = {"vv": [[ktt, ptt * utils.Du(ktt, sigmau_fiducial) ** 2]]}


# Fit true velocity interpolation sigma_u

In the model, there is a sigma_u parameter which can be interpreted to the smoothing of SNIa position due to peculiar velocities. \
To fit simultaneously this parameter, we interpolate the covariance matrix. 

In [4]:
sigmau_list = np.linspace(10.0, 20.0,20)[:, np.newaxis]
covariance_list = []
size_batch=10_000
number_worker=16


for sigu in sigmau_list:
    
    power_spectrum_dict = {"vv": [[ktt, ptt* utils.Du(ktt, sigu)**2]]}

    covariance_list.append(
        DataTrueVel.compute_covariance(
            "carreres23",
            power_spectrum_dict,
            size_batch=size_batch,
            number_worker=number_worker
        )
    )


[ 000007.75 ]: 11-24 15:33  root            INFO     Covariance matrix generated from flip with carreres23 model in 1.53e+00 seconds
[ 000009.41 ]: 11-24 15:33  root            INFO     Covariance matrix generated from flip with carreres23 model in 1.50e+00 seconds
[ 000011.15 ]: 11-24 15:33  root            INFO     Covariance matrix generated from flip with carreres23 model in 1.57e+00 seconds
[ 000012.82 ]: 11-24 15:33  root            INFO     Covariance matrix generated from flip with carreres23 model in 1.51e+00 seconds
[ 000014.52 ]: 11-24 15:33  root            INFO     Covariance matrix generated from flip with carreres23 model in 1.54e+00 seconds
[ 000016.24 ]: 11-24 15:33  root            INFO     Covariance matrix generated from flip with carreres23 model in 1.56e+00 seconds
[ 000017.42 ]: 11-24 15:33  root            INFO     Covariance matrix generated from flip with carreres23 model in 1.03e+00 seconds
[ 000018.61 ]: 11-24 15:33  root            INFO     Covariance matri

In [ ]:
covariance_list

In [ ]:
emulator_model = "gpmatrix"
emulator_parameter_names = ["sigu"]

covariance_gpmatrix = covariance.CovMatrix.init_from_emulator(
            emulator_model,
            "velocity",
            covariance_list,
            sigmau_list,
            emulator_parameter_names,
            verbose=True,
            test_emulator=False,
        )



In [ ]:
parameter_dict = {"fs8": {"value": 0.4,
                          "limit_low" : 0.0,
                          "fixed" : False,
                         },
                  "sigv": {"value": 200,
                          "limit_low" : 0.0,
                          "fixed" : False,
                         },
                  "sigu": {"value": 15.5,
                          "limit_low" : 10.0,
                          "limit_up" : 20.0,
                          "fixed" : False,
                         },
                 }


prior_dict = {"sigu": {"type": "gaussian",
                       "mean" : 15.0,
                       "standard_deviation" : 5.0,},
             }

likelihood_type="multivariate_gaussian"
likelihood_properties = {"inversion_method": "cholesky",
                         "prior": prior_dict}



minuit_fitter = fitter.FitMinuit.init_from_covariance(
        covariance_gpmatrix,
        DataTrueVel,
        parameter_dict,
        likelihood_type=likelihood_type,
        likelihood_properties = likelihood_properties,
)

print('Free parameters:', minuit_fitter.likelihood.free_par)

minuit_fitter.run()
